<a href="https://colab.research.google.com/github/AsenDulakshanaKavinda/recurrent-neural-network/blob/main/Sentiment_detection_with_LSTMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

'''
# Import kagglehub and download the dataset
import kagglehub
path = kagglehub.dataset_download("kazanova/sentiment140")
print("Path to dataset files:", path)

# Import shutil to move the files
import shutil
import os

# Define your target directory
target_dir = "/content/drive/MyDrive/DeepLearningProjects/SentimentDetectionData"

# Create the target directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Move the downloaded files to your desired location
shutil.move(path, target_dir)

print(f"Dataset moved to: {target_dir}") '''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'\n# Import kagglehub and download the dataset\nimport kagglehub\npath = kagglehub.dataset_download("kazanova/sentiment140")\nprint("Path to dataset files:", path)\n\n# Import shutil to move the files\nimport shutil\nimport os\n\n# Define your target directory\ntarget_dir = "/content/drive/MyDrive/DeepLearningProjects/SentimentDetectionData"\n\n# Create the target directory if it doesn\'t exist\nos.makedirs(target_dir, exist_ok=True)\n\n# Move the downloaded files to your desired location\nshutil.move(path, target_dir)\n\nprint(f"Dataset moved to: {target_dir}") '

In [ ]:
import kagglehub
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
from collections import Counter

In [ ]:
from tqdm import tqdm

# Enable tqdm for pandas
tqdm.pandas()

In [ ]:
import nltk
nltk.download('punkt_tab')  # Download punkt for tokenization
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
DEVICE = 'cpu ' # 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import pandas as pd

# Path to the file in your Google Drive
file_path = "/content/drive/MyDrive/DeepLearningProjects/SentimentDetectionData/data/training.1600000.processed.noemoticon.csv"

# Column names based on Sentiment140 documentation
column_names = [
    "target",    # sentiment (0 = negative, 2 = neutral, 4 = positive)
    "id",        # tweet ID
    "date",      # date of the tweet
    "flag",      # query flag (usually "NO_QUERY")
    "user",      # username
    "text"       # tweet text
]

# Read the CSV file
data = pd.read_csv(file_path,
                 names=column_names,  # Specify column names since the file has no header
                 encoding='latin-1')  # Use latin-1 encoding as the default UTF-8 might fail



In [ ]:
data_original = data.copy()

In [ ]:
data = data[['text', 'target']]
data['target'] = data['target'].replace(4, 1)

In [ ]:
# display sample of data
data.sample(10)

,text,target
765644,i got my first cavity,0
749049,I HATE this feeling of guilt i have when i hav...,0
693565,watch the #Lions game online! looks like the b...,0
9172,[-O] Soooo full! And surrounded by dragons fan...,0
1349807,All We Know is Falling no Brasil Depois de QU...,1
470337,"wearing green, but there is a 60% chance of t-...",0
1419814,.... READY. STEADY.. .. wait - http://is.gd/QKoN,1
328354,My work schedule=4 days a week.,0
1022674,@LivitLuvit Your boyfriend posts are my favori...,1
888936,Roy Jones Jr = Pur Boxeur,1


In [ ]:
data.isnull().sum()

,0
text,0
target,0


In [ ]:
print(data['target'].unique())
print(data['target'].value_counts())

[0 1]
target
0    800000
1    800000
Name: count, dtype: int64


In [ ]:
def clean_text(text):
  text = re.sub(r'http\S+|https\S+', '', text, flags=re.MULTILINE) # remove urls
  text = re.sub(r'@\w+|\#', '', text) # remove mentions and hashtags
  text = re.sub(r'[^\w\s]', '', text) # remove punctuation
  return text.lower().strip()

In [ ]:
data['text'] = data['text'].progress_apply(clean_text)

100%|██████████| 1600000/1600000 [00:12<00:00, 128758.25it/s]


In [ ]:
def tokenize(text):
  return word_tokenize(text)

In [ ]:
data['tokens'] = data['text'].progress_apply(tokenize)

100%|██████████| 1600000/1600000 [02:41<00:00, 9935.38it/s]


In [ ]:
data.sample(10)

,text,target,tokens
1414812,graduating today,1,"[graduating, today]"
1529849,i want a yummy apple,1,"[i, want, a, yummy, apple]"
1061041,i gain them on trending topics and watch them ...,1,"[i, gain, them, on, trending, topics, and, wat..."
1114584,hagfish are fun,1,"[hagfish, are, fun]"
155663,not feeling well,0,"[not, feeling, well]"
262978,jk ill take any of them,0,"[jk, ill, take, any, of, them]"
1233185,really wanting to do pilates lets work on thi...,1,"[really, wanting, to, do, pilates, lets, work,..."
1172799,is this new click the url go demi lovato loved,1,"[is, this, new, click, the, url, go, demi, lov..."
48110,just got home from dubbo and oh wow i am tireddd,0,"[just, got, home, from, dubbo, and, oh, wow, i..."
1277749,lol if youre going to talk rubbish you should ...,1,"[lol, if, youre, going, to, talk, rubbish, you..."


In [ ]:
def build_vocab(tokenize_texts, min_freq=2):
  word_counts = Counter()
  for tokens in tokenize_texts:
    word_counts.update(tokens)

  vocab = {'<unk>': 0, '<pad>': 1}
  idx = 2
  for word, freq in word_counts.items():
    if freq >= min_freq:
      vocab[word] = idx
      idx += 1
  return vocab

In [ ]:
vocab = build_vocab(tqdm(data['tokens'], desc="Building vocab"), min_freq=2)
print(f"Vocabulary size: {len(vocab)}")

Building vocab: 100%|██████████| 1600000/1600000 [00:07<00:00, 226061.62it/s]


Vocabulary size: 138146


In [ ]:
# Convert tokens to indices
def tokens_to_indices(tokens, vocab):
    return [vocab.get(token, vocab['<unk>']) for token in tokens]

data['text_indices'] = data['tokens'].progress_apply(lambda x: tokens_to_indices(x, vocab))

100%|██████████| 1600000/1600000 [00:07<00:00, 200296.50it/s]


In [ ]:
data.sample(10)

,text,target,tokens,text_indices
1159580,wondering why i got up so early and why my liv...,1,"[wondering, why, i, got, up, so, early, and, w...","[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
298019,sad i love you justin,0,"[sad, i, love, you, justin]","[0, 2, 3, 0, 0]"
757188,just received the income tax bill,0,"[just, received, the, income, tax, bill]","[0, 0, 0, 0, 0, 0]"
1575908,lvatt is out now and its trending i love im g...,1,"[lvatt, is, out, now, and, its, trending, i, l...","[0, 0, 0, 0, 0, 0, 0, 2, 3, 0, 0, 4, 0, 0, 0, ..."
1417281,oh man this is sooo beautiful i wanna do paint...,1,"[oh, man, this, is, sooo, beautiful, i, wan, n...","[0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1550014,quotmaybe years until the day my dreams would ...,1,"[quotmaybe, years, until, the, day, my, dreams...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
348499,today can be a really good day or a really bad...,0,"[today, can, be, a, really, good, day, or, a, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0]"
809015,had a wonderful dream about her future husband...,1,"[had, a, wonderful, dream, about, her, future,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, ..."
1564348,and thats also why i like you,1,"[and, thats, also, why, i, like, you]","[0, 0, 0, 0, 2, 0, 0]"
1328631,sushiii at 745 in the amm i am a feen,1,"[sushiii, at, 745, in, the, amm, i, am, a, feen]","[0, 0, 0, 0, 0, 0, 2, 0, 0, 0]"


In [ ]:
# Pad sequences
max_len = 50  # Max sequence length
sequences = [torch.tensor(seq[:max_len], dtype=torch.long) for seq in data['text_indices']]
padded_sequences = pad_sequence(sequences, batch_first=True, padding_value=vocab['<pad>'])

In [ ]:
# Convert targets to tensor
labels = torch.tensor(data['target'].values, dtype=torch.long)

In [ ]:
# Create dataset and dataloaders
dataset = TensorDataset(padded_sequences, labels)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)